# Experiments

## Introduction to the 1-D diffusion problem

I performed a variety of experiments of increasing complexity to test and exercise the neural network code for solving the 1-D diffusion problem ([`nnpde2diff1d.py`](../nnpde2diff1d.py)).

Initial work on this project used the standard delta algorithm with a fixed training rate $\eta$ (typically $\eta=0.1$) Further results obtained using the `minimize` function from the `scipy.optimize` Python package were superior in performance, precision, and accuracy. The BFGS minimization algorithm was found to be the best (from a speed and stability standpoint) of the options available in the `scipy.optimize.minimize` function. I tried all of the other algorithms, and all were successful to a greater or lesser extent, but based on the results here, I will use BFGS going forward. The [BFGS](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#broyden-fletcher-goldfarb-shanno-algorithm-method-bfgs) (Broyden-Fletcher-Goldfarb-Shanno) algorithm uses the analytical gradient of the error function rather than a numerically-estimated gradient.

Unless otherwise specified, an 11x11 evenly-spaced grid of training points in time and space was used (`nx = nt = 11`). The random number seed was set to 0 in most cases, for reproducibility. When a seed of 0 resulted in errors (typically numeric overflow), the seed was incremented by 1 until a suitable value was found, or until the seed reached the value of 9. In the latter case, the attempt was abandoned.

Uness otherwise stated, the diffusion coefficient was unity (`D=1`).

All analytical solutions were obtained using `DSolve` in Mathematica 11. Solutions were typically infinite Fourier series, so the series were arbitrarily terminated after 100 terms for this work.

All times were generated using Python 3.6.6 and numpy 1.15.2. Runs were performed in a Jupyter notebook on my MacBook Pro (macOS High Sierra v10.13.6, 2.8 GHz Intel Core I7, 16 GB RAM). Other activity on the machine during the runs was negligible. The current code has been optimized for readability and algorithm verification, not for speed.

This code is based on the work described in "Artificial Neural Networks for Solving Ordinary and Partial Differential Equations" (_IEEE Transactions on Neural Networks_ __9__(5), 1998), by Lagaris _et al_ (https://ieeexplore.ieee.org/document/712178).

## Set 1: Stability checks

### [diff1d_0_BFGS.ipynb](diff1d_0_BFGS.ipynb)

This is the simplest possible check. The initial profile is flat at `Y=0` from `x=0` to `x=1`. The BC at `x=0` and `x=1` are fixed at 0. Therefore, the profile should not change with time, and should remain flat at `Y=0`.

The initial run used the BFGS algorithm, with defaults for all parameters. The default convergence criterion was `gtol=1e-5`, where `gtol` is the "infinite norm" (the maximum value) of the gradient of the objective function. The absolute error was $O(10^{-6}$). Since the analytical solution was 0, the relative error was undefined. Interestingly, the error was highest ($O(3\times10^{-6})$) at the first time step (`t=0.1`), then oscillated about 0 as time proceeded.

I then decreased the value of `gtol` to determine how precise the final results could be. I examined the results with `gtol` values of $10^{-6}$ (absolute error O($10^{-6)}$)), $10^{-8}$ (absolute error O($6\times10^{-8}$)), $10^{-10}$ (absolute error O($10^{-9}$)), $10^{-12}$ (absolute error O($10^{-9}$)), and $10^{-14}$ (terminated due to precision loss, but final result had absolute error O($10^{-9}$).

Execution time ranged from 18 s to 584 s.

### [diff1d_1_BFGS.ipynb](diff1d_1_BFGS.ipynb)

This case was similar to [diff1d_0_BFGS.ipynb](diff1d_0_BFGS.ipynb), but the profile was fixed at `Y=1`. The trained network parameters and error behavior for this case were identical to those for the [diff1d_0_BFGS.ipynb](diff1d_0_BFGS.ipynb) case.

Execution time ranged from 17 s to 537 s.

### [diff1d_flat_BFGS.ipynb](diff1d_flat_BFGS)

This case was similar to [diff1d_0_BFGS.ipynb](diff1d_0_BFGS.ipynb), but the profile was fixed at `Y=0.5`. The trained network parameters and error behavior for this case were identical to those for the [diff1d_0_BFGS.ipynb](diff1d_0_BFGS.ipynb) case.

Execution time ranged from 16 s to 555 s.

### [diff1d_rampup_BFGS.ipynb](diff1d_rampup_BFGS.ipynb)

In this case, the initial profile was a linear ramp from `Y(0,0)=0` to `Y(1,0)=1`. The BC were fixed at 0 for `x=0`, 1 for `x=1`, and so the profile should not change with time. The trained network parameters and error behavior for this case were identical to those for the [diff1d_0_BFGS.ipynb](diff1d_0_BFGS.ipynb) case.

Execution time ranged from 16 s to 593 s.

### [diff1d_rampdown_BFGS.ipynb](diff1d_rampdown_BFGS.ipynb)

In this case, the initial profile was a linear ramp from `Y(0,0)=1` to `Y(1,0)=0`. The BC were fixed at 1 for `x=0`, 0 for `x=1`, and so the profile should not change with time. The trained network parameters and error behavior for this case were identical to those for the [diff1d_0_BFGS.ipynb](diff1d_0_BFGS.ipynb) case.

Execution time ranged from 16 s to 514 s.

## Set 2: Nonlinear, static BC cases

### [diff1d_sine_BFGS.ipynb](diff1d_sine_BFGS.ipynb)

This case was not run successfully due to overflow exceptions.

### [diff1d_flat+sine_BFGS.ipynb](diff1d_flat+sine_BFGS.ipynb)

This case used a starting profile consisting of a half-sine wave with amplitude of 0.5, based at `Y=0.5`:

`Y(x,0) = 0.5*(1 + sin(pi*x))`

The boundary conditions at `x=0` and 1 were fixed at 0.5. The expected behavior was a decay of the profile to become flat at `Y=0.5`. A different random number seed (3) was used, since the original value of 0 resulted in overflow exceptions.

The initial case using BFGS defaults ended when the maximum iteration count of 8000 was reached. Upon examination of the incomplete results, as in previous cases, the  error decreased rapidly over the first few time steps. The relative error was initially as high as 0.025, but by `t=0.4` was of O($10^{-3}$). The maximum iteration count was then increased to 16000, using the same default `gtol` of $10^{-5}$. In this case, the training terminated due to precision loss. These incomplete results were very similar to the previous case, which is not surprising given that the precision loss forced termination after 8146 iterations, as opposed to 8000 iterations in the previous case.

Execution time ranged from 1459 s to 1483 s.

### [diff1d_flat+sine_D1e-1_BFGS.ipynb](diff1d_flat+sine_D1e-1_BFGS.ipynb)

This case is similar to the previous case, the only difference being a diffusion coefficient `D=0.1` was used. Since the large-time solutions did not approach closely to 0, the training was more stable. Relative error for the default case of `gtol`=$10^{-5}$ was O($10^{-5}$), decreasing to O($10^{-6}$) for `gtol`=$10^{-6}$, $10^{-7}$ at `gtol`=$10^{-8}$, somewhat smaller at $10^{-10}$ (at which point the iteration limit was set to 32000 to ensure convergence). For `gtol`=$10^{-10}$, the error was O($10^{-7}$), despite early termination due to precision loss.

Execution time ranged from 35 s to 5298 s.

## Set 3: Time-varying BC cases

### [diff1d_increase_linear_BFGS.ipynb](diff1d_increase_linear_BFGS.ipynb)

The initial profile was flat at `Y=0`. The BC at `x=1` was fixed at 0, while the BC at `x=0` increased at a linear rate:

$\frac {dY}{dt}(x=0)=a$, with $a=0.5$

This BC caused convergence problems even at low `gtol` values, due to numerical overflow. Presumably this problem is similar to the error decay over time in previous cases, and is likely caused by the discontinuity in the higher-order derivatives of the boundary condition function at x=0.

With the exception of the first time step, absolute error was of O($10^{-4}$).

### [diff1d_increase_BFGS.ipynb](diff1d_increase_BFGS.ipynb)

The initial profile was flat at `Y=0`. The BC at `x=1` was fixed at 0, while the BC at `x=0` increased at an accelerating rate:

$\frac {dY}{dt}(x=0) = 0.5 a t^2$, with $a=1$

This BC was selected after repeated failures to obtain reasonable solutions with a linear increase at `x=0`.

With the exception of the first time step, absolute error was of O($10^{-5}$). The error decay after the first time step was much faster than observed in previous cases. The error improved to O($10^{-6}$) for `gtol`=$10^{-6}$, and did not improve significantly as `gtol` was increased, since the calculations terminated due to precision loss.

Execution time ranged from 189 s to 2328 s.

### [diff1d_decrease_linear_BFGS.ipynb](diff1d_decrease_linear_BFGS.ipynb)

This case was analogous to the [diff1d_increase_linear_BFGS.ipynb](diff1d_increase_linear_BFGS.ipynb) case, but the profile started flat at `Y=1` and decreased at `x=0`. The relative error was O($10^{-5}$) in the default case, and runs at smaller `gtol` values terminated due to precision loss.

### [diff1d_decrease_BFGS.ipynb](diff1d_decrease_BFGS.ipynb)

This case was analogous to the [diff1d_increase_BFGS.ipynb](diff1d_increase_BFGS.ipynb) case, but the profile started flat at `Y=1` and decreased at `x=0`. The error was again O($10^{-5}$) in the default case, with little or no improvement as `gtol` was refined and the maximum iteration count was increased.

Execution time ranged from 509 s to 5298 s.

### [diff1d_increase_21x21_BFGS.ipynb](diff1d_increase_21x21_BFGS.ipynb)

This case used the accelerating increase scenario, but a training point of 21x21 points. The relative error was O($10^{-5}$), decreasing to O($10^{-6}$) at `gtol`=$10^{-8}$.

### [diff1d_decrease_21x21_BFGS.ipynb](diff1d_decrease_21x21_BFGS.ipynb)

This case used the accelerating decrease scenario, but a training grid of 21x21 points. The relative error was performance was comparable to the previous case.

### [diff1d_decrease_41x41_BFGS.ipynb](diff1d_decrease_41x41_BFGS.ipynb)

This case used the accelerating decrease scenario, but a training grid of 41x41 points. The relative error was performance was comparable to the previous case, but the run terminated due to precision loss when `gtol` was only $10^{-6}$.

## Summary

The stabililty check cases ensured that the code did not introduce spurious behavior. I found it interesting that the 5 linear cases all resulted in exactly the same values for the network parameters. Since no dynamic behavior was expected, and all cases started with the same random seed value, perhaps this should not be surprising.

The static BC cases are now (finally) exhibiting the correct quantitative behavior. Earlier attempts at these cases behaved correectly qualitatively, but were quantitatively poor. The problem was traced to an error in the form of the trial solution used by the neural network. Once that error was corrected, the quantitative behavior was much improved.

Of course, the interesting part here is the cases with dynamic BC. These cases, as well as the static BC cases, exhibited the behavior of rapid decay in the solution error as time steps proceeded. As mentioned above, I believe this is caused by the discontinuity in the higher-order derivatives of the BC function. If I recall correctly, similar effects are observed in finite-difference solutions of this type.

## Ideas for next steps

* Investigate techniques for normalization of the weights $v_k$ at the output node to prevent numeric overflow. Restricting initial weights to a smaller range may also be useful.

* Examine accuracy of the solution beyond the $t=1$ boundary. No training points from this region were used, so this should be a useful measure of the accuracy of the solution.

* Enhance the code to handle the 2-D, and then 3-D, diffusion problems. The main code difference will be in the trial function, and the increased number of training points - the actual training code and computation of the objective function will remain the same as in the 1-D case.

* Investigate oscillatory behavior with time - will the spatial oscillation frequency increase as t > 1?